In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [6]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection /Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection /Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection /Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2443 images belonging to 2 classes.
Found 610 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [7]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
77/77 [==============================] - 719s 9s/step - loss: 0.4825 - accuracy: 0.8039 - val_loss: 0.2213 - val_accuracy: 0.9131
Epoch 2/5
77/77 [==============================] - 288s 4s/step - loss: 0.1768 - accuracy: 0.9374 - val_loss: 0.2299 - val_accuracy: 0.9197
Epoch 3/5
77/77 [==============================] - 288s 4s/step - loss: 0.0978 - accuracy: 0.9664 - val_loss: 0.0374 - val_accuracy: 0.9885
Epoch 4/5
77/77 [==============================] - 280s 4s/step - loss: 0.0461 - accuracy: 0.9873 - val_loss: 0.0175 - val_accuracy: 0.9967
Epoch 5/5
77/77 [==============================] - 301s 4s/step - loss: 0.0138 - accuracy: 0.9955 - val_loss: 0.0204 - val_accuracy: 0.9934


In [10]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
from tensorflow.keras.preprocessing import image
#load the saved model
model = load_model('/content/Model.h5')
#Load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection /Brain_Tumor_Detection/test/pred/pred17.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)#add barch dimension
img_array/=255. #Normalize the pixel values
#Make predictions
prediction=model.predict(img_array)
#print the prediction
if prediction<0.5:
  print("Prediction : No tumour (Probability :",prediction[0][0],")")
else:
  print("Prediction : tumour present (Probability :",prediction[0][0],")")
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

1/1 [==============================] - 0s 138ms/step
Prediction : tumour present (Probability : 0.64216286 )
